# Population density data
I found the shapefiles of the neighborhood of Turin here:

http://geoportale.comune.torino.it/geodati/zip/zonestat_popolazione_residente_geo.zip


I found the data on number of residents by neighborhood of 2020 here:

http://aperto.comune.torino.it/en/dataset/popolazione-per-sesso-e-zona-statistica-2020

In this notebook I combine in one file the geometry of the first file with the population data of the second one. From these data I also compute the density of population by neighborhood.

## Setup

In [ ]:
%run -i path.py
%run -i setup.py

In [ ]:
# gdf of the edges of Turin street network
gdf_edges = gpd.read_file(PATH['data']+'graph_shapefile/Turin/edges.shp')

In [ ]:
# pop data (zone statistiche) 2020

# load the shapefile
pop_shp = gpd.read_file(PATH["data"]+'zonestat_popolazione_residente_2007_geo.shp')

# load the population data
pop_csv = pd.read_csv(PATH["data"]+'Torino_pop_zone-statistiche.csv')

In [ ]:
pop_shp.crs

In [ ]:
pop_shp = pop_shp.to_crs(4326)

In [ ]:
per_Area = gpd.read_file(PATH["data"]+'zonestat_popolazione_residente_2007_geo.shp')

per_Area.crs

In [ ]:
# we need to keep crs= EPSG:3003 to calculate the area of the polygons
per_Area = per_Area.sort_values(by='ZONA_STATI')

In [ ]:
pop_shp = pop_shp.sort_values(by='ZONA_STATI')
pop_shp

In [ ]:
# calculate population density 2020 and add it to 'pop_shp'

dens_f = []
dens_m = []
dens_t = []
for i in range(len(pop_csv)):
    dens_f.append(pop_csv['Femmine'].iloc[i]/(per_Area['geometry'].iloc[i].area/1e6))
    dens_m.append(pop_csv['Maschi'].iloc[i]/(per_Area['geometry'].iloc[i].area/1e6))
    dens_t.append((pop_csv['Maschi'].iloc[i]+pop_csv['Femmine'].iloc[i])/(per_Area['geometry'].iloc[i].area/1e6))

pop_shp['dens_fpop'] = dens_f
pop_shp['dens_mpop'] = dens_m
pop_shp['dens_totpop'] = dens_t

pop_shp

In [ ]:
# create a column in the pop_csv df with the number that identify the 'zona statistica'

n_zonastat = []

for ii in range(len(pop_csv)):
    zona = pop_csv['Zona_Statistica'].iloc[ii]
    for i in range(len(zona)):
        if zona[i] == ' ':
            n_zonastat.append(zona[:i])
            break

pop_csv['ZONASTAT'] = n_zonastat

In [ ]:
# remove the number of residents of 2007 (we keep the geometry from this file and the number of residents from pop_csv)
pop_shp = pop_shp.drop(['NMASCHI','NFEMMINE','TOTALE','ANNO'],axis=1)

In [ ]:
# add the number of resident of 2020 in pop_shp

pop = []
popF = []
popM = []


for i in range(len(pop_shp)):
    pop.append(pop_csv.iloc[i]['Femmine']+pop_csv.iloc[i]['Maschi'])
    popF.append(pop_csv.iloc[i]['Femmine'])
    popM.append(pop_csv.iloc[i]['Maschi'])
    
pop_shp['tot_pop'] = pop
pop_shp['female_pop'] = popF
pop_shp['male_pop'] = popM

In [ ]:
# plot the density on the map

fig, axes = plt.subplots(nrows=1,ncols=1, figsize=(25,25), sharex=True, sharey=True)

matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20)

divider = make_axes_locatable(axes)

axes.set_title('Residents in Turin',fontsize=40)
        
cax = divider.append_axes('right', size='2%', pad=0.6)
cax.set_xlabel('Pop. density',labelpad=20,size='xx-large')
axes = pop_shp.plot(column='dens_totpop',cmap='OrRd', legend=True,ax=axes, cax=cax, alpha = .9)

gdf_edges.plot(ax = axes,alpha=.2, color='black')

In [ ]:
pop_shp.to_file(driver = 'ESRI Shapefile', filename = PATH["data"]+ placeid + "/"+'popTorino2020')